In [2]:
# Download the Custom Criminal Indicator Data
!gdown "1VgzwriksHMj5uiX2pYk7cv8hOavA1fku"

Downloading...
From: https://drive.google.com/uc?id=1VgzwriksHMj5uiX2pYk7cv8hOavA1fku
To: /content/criminal_indicator.csv
100% 6.86M/6.86M [00:00<00:00, 36.9MB/s]


In [3]:
# Download the Action Recommendation Dataset
!gdown "1ECcRj3u6g04z-4ODNSHoJC3jRz6ba0nd"

Downloading...
From (original): https://drive.google.com/uc?id=1ECcRj3u6g04z-4ODNSHoJC3jRz6ba0nd
From (redirected): https://drive.google.com/uc?id=1ECcRj3u6g04z-4ODNSHoJC3jRz6ba0nd&confirm=t&uuid=2eb7f999-6fef-4761-93e4-a7d407b0d2c4
To: /content/dataset.npy
100% 487M/487M [00:04<00:00, 115MB/s]


In [4]:
import numpy as np

with open("/content/dataset.npy", "rb") as f:
  dataset = np.load(f, allow_pickle=True)

len(dataset)

220065

In [9]:
cities = list(set([row['municipio'].upper() for row in dataset]))
len(cities)

322

In [16]:
import pandas as pd

df = pd.read_csv("/content/criminal_indicator.csv")
df = df[df['municipio_norm'].isin(cities)]
len(df['municipio_norm'].unique())

322

In [17]:
df

,municipio_norm,codigo_municipio,uf,ano,semestre,vitimas_homicidio_doloso,populacao,taxa_homicidios_100k
946,CAMPESTRE,2701357.0,AL,2015,1,0,6665.0,0.000000
947,CAMPESTRE,3111002.0,AL,2015,1,0,20696.0,0.000000
948,CAMPESTRE,2701357.0,AL,2015,2,1,6665.0,15.003751
949,CAMPESTRE,3111002.0,AL,2015,2,1,20696.0,4.831852
950,CAMPESTRE,2701357.0,AL,2016,1,2,6665.0,30.007502
...,...,...,...,...,...,...,...,...
136799,RIO SONO,1718758.0,TO,2023,2,0,4841.0,0.000000
136800,RIO SONO,1718758.0,TO,2024,1,0,4841.0,0.000000
136801,RIO SONO,1718758.0,TO,2024,2,1,4841.0,20.656889
136802,RIO SONO,1718758.0,TO,2025,1,0,4841.0,0.000000


In [19]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Carrega o modelo uma única vez (fora da função, para não recarregar toda hora)
model = SentenceTransformer("embaas/sentence-transformers-multilingual-e5-base")

def top_k(text: str,
          embeddings: np.ndarray,
          k: int = 5) -> np.ndarray:
    """
    Retorna os índices das k embeddings mais similares semanticamente ao `texto`.

    Parâmetros
    ----------
    text : str
        Texto de entrada.
    embeddings : np.ndarray
        Matriz de embeddings com shape (N, 768), por exemplo (250000, 768).
        Idealmente pré-computadas com o mesmo modelo.
    k : int
        Número de vizinhos mais próximos a retornar.

    Retorno
    -------
    np.ndarray
        Array 1D com os índices das k embeddings mais similares.
    """

    # 1) Codificar o texto de entrada em um embedding
    # Para modelos E5, é comum prefixar com "query: "
    consulta = f"query: {text}"
    embedding_texto = model.encode(
        consulta,
        normalize_embeddings=True  # já normaliza o vetor (norma L2 = 1)
    )  # shape (768,)

    # 2) Garantir que a matriz de embeddings também está normalizada
    #    (se você já salvou normalizada, pode pular esse passo)
    normas = np.linalg.norm(embeddings, axis=1, keepdims=True)
    # Evitar divisão por zero
    normas[normas == 0] = 1e-12
    embeddings_norm = embeddings / normas

    # 3) Similaridade por produto interno (equivale a cosseno se tudo está normalizado)
    #    resultado: vetor de similaridades shape (N,)
    scores = embeddings_norm @ embedding_texto

    # 4) Pegar os índices das k maiores similaridades
    # argpartition é mais eficiente que sort para top-k
    if k >= len(scores):
        # Se k >= N, só ordena tudo
        return np.argsort(-scores)

    idx_part = np.argpartition(-scores, k)[:k]  # k maiores (desordenados)
    # Ordenar esses k pelo score decrescente
    idx_ordenados = idx_part[np.argsort(-scores[idx_part])]

    return idx_ordenados

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [25]:
dataset[10]

{'sapl_base': 'https://sapl.maraba.pa.leg.br',
 'sapl_url': 'https://sapl.maraba.pa.leg.br',
 'municipio': 'Marabá',
 'uf': 'PA',
 'tipo_id': 9,
 'tipo_label': 'ANT Anteprojeto de Lei ',
 'materia_id': 27379,
 'numero': 12,
 'ano': 2025,
 'ementa': 'ANTEPROJETO DE LEI Nº 01 DE 10 DE ABRIL 2025\r\nINSTITUI  E REGULAMENTA A IMPLANTAÇÃO DA CASA DE PARTO  NO MUNICÍPIO DE MARABÁ, E DÁ OUTRAS PROVIDÊNCIAS.',
 'data_apresentacao': '2025-04-10',
 'em_tramitacao': True,
 'situacao': None,
 'link_publico': 'https://sapl.maraba.pa.leg.br/materia/27379/acompanhar-materia/',
 'acao': 'Instituir e regulamentar a implantação da Casa de Parto no município.',
 'embedding': array([ 0.006138 ,  0.01941  , -0.005856 ,  0.047    ,  0.02234  ,
        -0.00502  , -0.0306   , -0.0444   ,  0.002518 ,  0.02133  ,
        -0.0491   ,  0.007385 ,  0.1006   ,  0.0452   , -0.06537  ,
        -0.03192  ,  0.001022 , -0.003258 ,  0.03323  ,  0.0008044,
         0.0185   , -0.0629   ,  0.0708   , -0.0175   ,  0.05655

In [20]:
embeddings = [row['embedding'] for row in dataset]
embeddings = np.concat([embeddings])

embeddings.shape

(220065, 768)

In [79]:
text = "Como diminuir a criminalidade no município?"
k = 1000

indexes_top_k = top_k(text, embeddings, k)

In [80]:
results = dataset[indexes_top_k]
results = [(row['municipio'], row['data_apresentacao'], row['acao']) for row in results]

pd.DataFrame(
    results,
    columns=['nome', 'data', 'acao']
)

,nome,data,acao
0,Caruaru,2023-06-15,"Implementar programa de prevenção às drogas, d..."
1,Guarujá do Sul,2021-09-30,Criar o programa “Juro Zero” no município.
2,Campina Grande,2025-09-11,Criar o programa municipal “Dignidade em Foco”...
3,Natal,2018-06-06,Criar o programa “Descarte Consciente” no muni...
4,Caruaru,2025-03-11,Criar o programa “Mulheres Sem Violência” no m...
...,...,...,...
995,Natal,2018-02-20,Instalar faixas de travessia de pedestres em f...
996,Araxá,2025-10-14,"Criar o programa “Doe uma Praça, Praça Esporti..."
997,São Sebastião do Caí,2025-09-30,Criar programa municipal de prevenção ao green...
998,Montenegro,2024-07-10,Criar programa municipal de prevenção ao green...


In [81]:
uf2score = {}

for idx, row in df.iterrows():
  key = (row['municipio_norm'], row['ano'], row['semestre'])
  uf2score[key] = row['taxa_homicidios_100k']

uf2score

{('CAMPESTRE', 2015, 1): 4.831851565519907,
 ('CAMPESTRE', 2015, 2): 4.831851565519907,
 ('CAMPESTRE', 2016, 1): 0.0,
 ('CAMPESTRE', 2016, 2): 0.0,
 ('CAMPESTRE', 2017, 1): 0.0,
 ('CAMPESTRE', 2017, 2): 9.663703131039814,
 ('CAMPESTRE', 2018, 1): 0.0,
 ('CAMPESTRE', 2018, 2): 0.0,
 ('CAMPESTRE', 2019, 1): 9.663703131039814,
 ('CAMPESTRE', 2019, 2): 0.0,
 ('CAMPESTRE', 2020, 1): 4.831851565519907,
 ('CAMPESTRE', 2020, 2): 0.0,
 ('CAMPESTRE', 2021, 1): 0.0,
 ('CAMPESTRE', 2021, 2): 4.831851565519907,
 ('CAMPESTRE', 2022, 1): 0.0,
 ('CAMPESTRE', 2022, 2): 0.0,
 ('CAMPESTRE', 2023, 1): 0.0,
 ('CAMPESTRE', 2023, 2): 4.831851565519907,
 ('CAMPESTRE', 2024, 1): 4.831851565519907,
 ('CAMPESTRE', 2024, 2): 0.0,
 ('CAMPESTRE', 2025, 1): 0.0,
 ('CAMPESTRE', 2025, 2): 0.0,
 ('CANARANA', 2015, 1): 0.0,
 ('CANARANA', 2015, 2): 0.0,
 ('CANARANA', 2016, 1): 11.601825353855672,
 ('CANARANA', 2016, 2): 0.0,
 ('CANARANA', 2017, 1): 7.734550235903781,
 ('CANARANA', 2017, 2): 27.07092582566324,
 ('CANARANA

In [82]:
from datetime import datetime

def encode_date(date_str: str):
    date = datetime.strptime(date_str, "%Y-%m-%d")
    year = date.year
    semester = 1 if date.month <= 6 else 2
    return year, semester


In [83]:
filter = []

for name, date, action in results:
  year, semester = encode_date(date)
  key = (name.upper(), year, semester)

  if semester == 1:
    next_key = (name.upper(), year, 2)
  else:
    next_key = (name.upper(), year + 1, 1)

  if next_key not in uf2score:
    continue

  if key in uf2score:
    delta = uf2score[next_key] - uf2score[key]
    filter.append((name, action, delta))


In [84]:
import unicodedata
import re
from statistics import mean, stdev
from typing import List, Tuple, Dict, Any, Optional


# ---------- 1. Pré-processamento de texto ----------

STOPWORDS = {
    "a", "as", "o", "os", "um", "uma", "uns", "umas",
    "de", "do", "da", "dos", "das",
    "em", "no", "na", "nos", "nas",
    "para", "pra", "pro", "por",
    "ao", "aos", "à", "às",
    "e"
}


def normalize_and_tokenize(text: str) -> List[str]:
    """
    Normaliza o texto e retorna tokens de conteúdo:
    - minúsculas
    - remove acentos
    - remove pontuação
    - quebra em tokens
    - remove stopwords
    """
    text = text.lower()

    # remove acentos
    text = unicodedata.normalize("NFD", text)
    text = "".join(ch for ch in text if not unicodedata.combining(ch))

    # remove pontuação
    text = re.sub(r"[^\w\s]", " ", text)

    # colapsa espaços
    text = re.sub(r"\s+", " ", text).strip()

    tokens = text.split()
    tokens = [t for t in tokens if t not in STOPWORDS]

    return tokens


# ---------- 2. Similaridade e agrupamento ----------

def jaccard_similarity(tokens_a: List[str], tokens_b: List[str]) -> float:
    """
    Similaridade de Jaccard entre conjuntos de tokens.
    """
    set_a = set(tokens_a)
    set_b = set(tokens_b)
    if not set_a and not set_b:
        return 1.0
    inter = len(set_a & set_b)
    uni = len(set_a | set_b)
    return inter / uni


def group_bills_by_structure(
    bills: List[Tuple[str, str, float]],
    threshold: float = 0.75,
) -> List[Dict[str, Any]]:
    """
    Agrupa projetos de lei (municipio, frase, score) em grupos
    estruturalmente similares, usando Jaccard sobre tokens normalizados.

    Retorna uma lista de grupos, cada grupo é um dict com:
      - "rep_tokens": tokens do representante
      - "rep_phrase": frase representante (primeira do grupo)
      - "members": lista de (municipio, frase, score, similarity)
    """
    if not bills:
        return []

    groups: List[Dict[str, Any]] = []

    for municipio, frase, score in bills:
        tokens = normalize_and_tokenize(frase)

        if not groups:
            groups.append({
                "rep_tokens": tokens,
                "rep_phrase": frase,
                "members": [(municipio, frase, score, 1.0)],
            })
            continue

        best_idx: Optional[int] = None
        best_sim = 0.0

        for i, g in enumerate(groups):
            sim = jaccard_similarity(tokens, g["rep_tokens"])
            if sim > best_sim:
                best_sim = sim
                best_idx = i

        if best_idx is not None and best_sim >= threshold:
            groups[best_idx]["members"].append(
                (municipio, frase, score, best_sim)
            )
        else:
            groups.append({
                "rep_tokens": tokens,
                "rep_phrase": frase,
                "members": [(municipio, frase, score, 1.0)],
            })

    return groups


# ---------- 3. Estatística básica ----------

def compute_mean_and_std(values: List[float]) -> Tuple[float, float]:
    """
    Calcula média e desvio padrão amostral.
    Se houver apenas um valor, desvio padrão = 0.0.
    """
    if not values:
        return 0.0, 0.0
    if len(values) == 1:
        return values[0], 0.0
    return mean(values), stdev(values)


# ---------- 4. Teste não paramétrico (Wilcoxon) ----------

def wilcoxon_less_than_zero(values: List[float], alpha: float = 0.05) -> Dict[str, Any]:
    """
    Aplica o teste de Wilcoxon signed-rank (não paramétrico)
    para verificar se a mediana dos valores é menor que zero.

    Retorna um dict:
      - "test": nome do teste
      - "p_value": p-valor (ou None se não foi possível calcular)
      - "reject_h0": True se rejeita H0 (mediana >= 0) em favor de mediana < 0
    """
    # Remove zeros (não contribuem para Wilcoxon)
    filtered = [v for v in values if v != 0.0]

    # Se poucos dados, o teste não faz muito sentido.
    if len(filtered) < 1:
        return {
            "test": "wilcoxon_signed_rank",
            "p_value": None,
            "reject_h0": False,
        }

    try:
        from scipy.stats import wilcoxon

        # H0: mediana = 0 ; H1: mediana < 0
        stat, p_value = wilcoxon(filtered, alternative="less", zero_method="wilcox")
        reject = (p_value is not None) and (p_value < alpha)
        return {
            "test": "wilcoxon_signed_rank",
            "p_value": float(p_value),
            "reject_h0": bool(reject),
        }
    except Exception:
        # Fallback simples: sem scipy, usa só o sinal da média
        m = mean(values)
        return {
            "test": "simple_mean_sign",
            "p_value": None,
            "reject_h0": bool(m < 0),
        }


# ---------- 5. Métrica de qualidade da política ----------

def compute_policy_quality(scores: List[float]) -> float:
    """
    Métrica de qualidade da política.

    Objetivo: políticas que reduzem o indicador (scores negativos)
    devem receber pontuação maior.

    A métrica considera:
      1) A proporção de municípios com score negativo (fraction_neg).
      2) A magnitude do efeito médio (mais negativa = melhor).

    Definição:
        fraction_neg = (# scores < 0) / n
        effect_mean  = mean(scores)

        quality = max(0, fraction_neg * (-effect_mean))

    Intuição:
      - Se poucos municípios melhoram (fraction_neg baixo), qualidade cai.
      - Se a média é pouco negativa (melhora fraca) ou positiva (piorou),
        a qualidade se aproxima de 0.
      - Positive outliers puxam a média pra cima e reduzem fraction_neg,
        penalizando políticas que causam piora em muitos lugares.
    """
    if not scores:
        return 0.0

    n = len(scores)
    neg_count = sum(1 for s in scores if s < 0)
    fraction_neg = neg_count / n if n > 0 else 0.0
    effect_mean = mean(scores)
    quality = fraction_neg * (-effect_mean)

    return max(0.0, quality)


# ---------- 6. Função principal: gerar políticas ----------

def generate_policies_from_bills(
    bills: List[Tuple[str, str, float]],
    similarity_threshold: float = 0.75,
    alpha: float = 0.05,
) -> List[Dict[str, Any]]:
    """
    Recebe uma lista de projetos de lei no formato:
      (municipio, descricao_PL, effect_score)

    Agrupa projetos estruturalmente similares e retorna uma lista de objetos:

      {
          "policy": Frase representativa do grupo,
          "actions": [(NomeMunicipio, Descricao, score), ...],
          "effect_mean": média dos scores,
          "effect_std": desvio padrão dos scores,
          "effective": resultado do teste não paramétrico
                       (dict com test, p_value, reject_h0),
          "quality_score": métrica que combina
                           (proporção de efeitos negativos) x
                           (magnitude média da melhoria)
      }
    """
    groups = group_bills_by_structure(bills, threshold=similarity_threshold)
    policies: List[Dict[str, Any]] = []

    for g in groups:
        members = g["members"]
        scores = [s for (_, _, s, _) in members]

        effect_mean, effect_std = compute_mean_and_std(scores)
        test_result = wilcoxon_less_than_zero(scores, alpha=alpha)
        quality_score = compute_policy_quality(scores)

        # agora actions inclui o score também
        actions = [(mun, frase, score) for (mun, frase, score, _) in members]

        policy_obj = {
            "policy": g["rep_phrase"],
            "actions": actions,
            "effect_mean": effect_mean,
            "effect_std": effect_std,
            "effective": test_result,
            "quality_score": quality_score,
        }
        policies.append(policy_obj)

    return policies


In [85]:
policies = generate_policies_from_bills(filter)
policies = sorted(policies, key=lambda x: x['quality_score'], reverse=True)

for p in policies:

    if len(p['actions']) <= 1:
      continue

    print("Policy:", p["policy"])
    print("  effect_mean:", p["effect_mean"])
    print("  effect_std:", p["effect_std"])
    print("  quality:", p["quality_score"])
    print("  effective:", p["effective"])
    print("  actions:")
    for mun, desc, score in p["actions"]:
        print(f"    - {mun}: {score:.3f} | {desc}")
    print()

Policy: Instituir o Prêmio “Advocacia Cidadã” no município.
  effect_mean: -20.167865956757787
  effect_std: 21.877958464530973
  quality: 20.167865956757787
  effective: {'test': 'wilcoxon_signed_rank', 'p_value': 0.25, 'reject_h0': False}
  actions:
    - Primavera do Leste: -4.698 | Instituir o Prêmio “Advocacia Cidadã” no município.
    - Dilermando de Aguiar: -35.638 | Instituir o Prêmio “Advocacia Cidadã” no município.

Policy: Criar o programa “Doe um Bicicletário” no município.
  effect_mean: -13.3505631006027
  effect_std: 24.061643366348033
  quality: 8.9003754004018
  effective: {'test': 'wilcoxon_signed_rank', 'p_value': 0.375, 'reject_h0': False}
  actions:
    - Cidreira: -41.005 | Criar o programa “Doe um Bicicletário” no município.
    - Pelotas: -1.842 | Criar o programa “Doe um Bicicletário” no município.
    - Itaqui: 2.796 | Criar o programa “Doe um Bicicletário” no município.

Policy: Criar o programa municipal “Prefeito Mirim” no município.
  effect_mean: -7.86287